### Not implementing this, but if there's need for binning in the future, this is a good reference

In [1]:
import arcpy
from arcpy import env
import os
import numpy as np
from arcgis import GIS
from arcgis.features import GeoAccessor
from arcgis.features import GeoSeriesAccessor
import pandas as pd
import jenkspy

arcpy.env.overwriteOutput = True
arcpy.env.parallelProcessingFactor = "90%"

# show all columns
pd.options.display.max_columns = None

# pd.DataFrame.spatial.from_featureclass(???)  
# df.spatial.to_featureclass(location=???,sanitize_columns=False)  

# gsa = arcgis.features.GeoSeriesAccessor(df['SHAPE'])  
# df['AREA'] = gsa.area  # KNOW YOUR UNITS

In [2]:
# fill NA values in Spatially enabled dataframes (ignores SHAPE column)
def fill_na_sedf(df_with_shape_column, fill_value=0):
    if 'SHAPE' in list(df_with_shape_column.columns):
        df = df_with_shape_column.copy()
        shape_column = df['SHAPE'].copy()
        del df['SHAPE']
        return df.fillna(fill_value).merge(shape_column,left_index=True, right_index=True, how='inner')
    else:
        raise Exception("Dataframe does not include 'SHAPE' column")

In [3]:
if not os.path.exists('Outputs'):
    os.makedirs('Outputs')
    
outputs = ['.\\Outputs', "scratch.gdb", 'results.gdb']
gdb = os.path.join(outputs[0], outputs[1])
gdb2 = os.path.join(outputs[0], outputs[2])

if not arcpy.Exists(gdb):
    arcpy.CreateFileGDB_management(outputs[0], outputs[1])

if not arcpy.Exists(gdb2):
    arcpy.CreateFileGDB_management(outputs[0], outputs[2])

In [4]:
# geogs = ['CENTER','CITYAREA', 'COUNTY', 'DISTMED', 'OTHERAREA', 'TAZ']
# categories = ['TPCL', 'HH', 'HJI', 'IND', 'OTHR', 'RTL', 'POP']
# metric = ['density', 'count']
# forcastorchange = ['change', 'forecast', 'comparechange', 'compareforecast']
# absoluteorpercent = ['absolute', 'percent']
# units = {'TPCL':'employees', 'HH':'households', 'HJI':'households x 1.8 + jobs', 'IND':'employees', 'OTHR':'employees', 'RTL':'employees', 'POP':'population'}
# colors = {'forecast':{'TPCL':['white','darkslateblue'], 'HH':['white','darkgoldenrod'], 'HJI':['white','darkgreen'], 'IND':['white','magenta'], 'OTHR':['white','royalblue'], 'RTL':['white','maroon'], 'POP':['white','goldenrod']}, 
#           'change':['lightblue','darkred'], 
#           'comparechange':['lightblue','darkred'], 
#           'compareforecast':{'TPCL':['white','darkslateblue'], 'HH':['white','darkgoldenrod'], 'HJI':['white','darkgreen'], 'IND':['white','magenta'], 'OTHR':['white','royalblue'], 'RTL':['white','maroon'], 'POP':['white','goldenrod']}}

In [373]:
# Scales a provided number according to UDOT standards
# Useful for class breaks and legends
# Fun fact: even preceding numbers round down from 5, ex 25 -> 20
def udot_round(n):
    if n >= .001 and n <= .0999:
        base = .25
        return base * round(float(n)/base, 1)
    
    if n >= .01 and n <= .999:
        base = .5
        return base * round(float(n)/base, 1)
    
    if n >= 1 and n <= 9.9:
        base = .5
        return base * round(float(n)/base)
    
    if n >= 10 and n <= 99:
        base = 10
        return base * round(float(n)/base)
    
    if n >= 100 and n <= 999:
        base = 50
        return int(base * round(float(n)/base))
    
    if n >= 1000 and n <= 9999:
        base = 100
        return int(base * round(float(n)/base))
    
    if n >= 10000 and n <= 99999:
        base = 500
        return int(base * round(float(n)/base))
    
    if n >= 100000 and n <= 999999:
        base = 1000
        return int(base * round(float(n)/base))
    else:
        return n

In [232]:
def quantile_mins(_df, _index, _density_mode=False, _scale=True):
    data_cols = [c for c in list(_df.columns) if c.startswith('N_') |  c.startswith('O_')]
    
    if _density_mode == True:
        for c in data_cols:
            _df[c] = round(_df[c] / _df['DEVACRES'] , 2)

    melted = pd.melt(_df, id_vars =[_index], value_vars=data_cols, var_name ='Year', value_name ='Value')
    melted['quantile'] = pd.qcut(melted['Value'].dropna(), duplicates='drop', q=6)
    bins = melted.groupby('quantile', as_index=False)[['Value']].min()
    if _scale == True:
        bins['Value'] = bins['Value'].apply(udot_round)
    return bins

In [375]:
udot_round(1.17)

1.0

In [309]:
def jenks_mins(_df, _index, _density_mode=False, _scale=True):
    data_cols = [c for c in list(_df.columns) if c.startswith('N_') |  c.startswith('O_')]

    if _density_mode == True:
        for c in data_cols:
            _df[c] = round(_df[c] / _df['DEVACRES'] , 2)

    melted = pd.melt(_df, id_vars =[_index], value_vars=data_cols, var_name ='Year', value_name ='Value')
    breaks = jenkspy.jenks_breaks(melted['Value'], n_classes=7)
    melted['jenks'] = pd.cut(melted['Value'],
                                 bins=breaks,
                                 labels=['bucket_1', 'bucket_2', 'bucket_3', 'bucket_4', 'bucket_5', 'bucket_6', 'bucket_7'],
                                 include_lowest=True)
    bins = melted.groupby('jenks', as_index=False)[['Value']].min()
    if _scale == True:
        bins['Value'] = bins['Value'].apply(udot_round)
    return bins

## CITYAREA

In [420]:
# HH
df = pd.DataFrame.spatial.from_featureclass('.\Outputs\map\CITYAREA.gdb\HH_PROJECTIONS_by_CITYAREA')
jenks_mins(df,'CityArea')

,jenks,Value
0,bucket_1,0.0
1,bucket_2,4300.0
2,bucket_3,11000.0
3,bucket_4,19500.0
4,bucket_5,29500.0
5,bucket_6,41500.0
6,bucket_7,81500.0


In [421]:
# POP
df = pd.DataFrame.spatial.from_featureclass('.\Outputs\map\CITYAREA.gdb\POP_PROJECTIONS_by_CITYAREA')
jenks_mins(df,'CityArea')

,jenks,Value
0,bucket_1,0.0
1,bucket_2,9800.0
2,bucket_3,27000.0
3,bucket_4,48500.0
4,bucket_5,79500.0
5,bucket_6,118000.0
6,bucket_7,191000.0


## CENTERS

In [318]:
# HH
df = pd.DataFrame.spatial.from_featureclass('.\Outputs\map\CENTER.gdb\HH_PROJECTIONS_by_CENTER')
jenks_mins(df,'CenterName', True, True)

,jenks,Value
0,bucket_1,0.0
1,bucket_2,0.0
2,bucket_3,2.5
3,bucket_4,5.0
4,bucket_5,7.5
5,bucket_6,10.0
6,bucket_7,20.0


In [319]:
# HJI
df = pd.DataFrame.spatial.from_featureclass('.\Outputs\map\CENTER.gdb\HJI_PROJECTIONS_by_CENTER')
jenks_mins(df,'CenterName', True, True)

,jenks,Value
0,bucket_1,0.0
1,bucket_2,7.5
2,bucket_3,20.0
3,bucket_4,30.0
4,bucket_5,40.0
5,bucket_6,60.0
6,bucket_7,100.0


In [335]:
# IND
df = pd.DataFrame.spatial.from_featureclass('.\Outputs\map\CENTER.gdb\IND_PROJECTIONS_by_CENTER')
jenks_mins(df, 'CenterName', True, True)

,jenks,Value
0,bucket_1,0.0
1,bucket_2,0.5
2,bucket_3,2.5
3,bucket_4,2.5
4,bucket_5,5.0
5,bucket_6,7.5
6,bucket_7,10.0


In [336]:
# OTHR
df = pd.DataFrame.spatial.from_featureclass('.\Outputs\map\CENTER.gdb\OTHR_PROJECTIONS_by_CENTER')
jenks_mins(df, 'CenterName', True, True)

,jenks,Value
0,bucket_1,0.0
1,bucket_2,5.0
2,bucket_3,10.0
3,bucket_4,20.0
4,bucket_5,30.0
5,bucket_6,50.0
6,bucket_7,80.0


In [337]:
# POP
df = pd.DataFrame.spatial.from_featureclass('.\Outputs\map\CENTER.gdb\POP_PROJECTIONS_by_CENTER')
jenks_mins(df, 'CenterName',True, True)

,jenks,Value
0,bucket_1,0.0
1,bucket_2,2.5
2,bucket_3,7.5
3,bucket_4,10.0
4,bucket_5,20.0
5,bucket_6,30.0
6,bucket_7,50.0


In [342]:
# RTL
df = pd.DataFrame.spatial.from_featureclass('.\Outputs\map\CENTER.gdb\RTL_PROJECTIONS_by_CENTER')
jenks_mins(df, 'CenterName', True, True)

,jenks,Value
0,bucket_1,0.0
1,bucket_2,2.5
2,bucket_3,5.0
3,bucket_4,10.0
4,bucket_5,20.0
5,bucket_6,30.0
6,bucket_7,80.0


In [343]:
# TPCL
df = pd.DataFrame.spatial.from_featureclass('.\Outputs\map\CENTER.gdb\TPCL_PROJECTIONS_by_CENTER')
jenks_mins(df, 'CenterName', True, True)

,jenks,Value
0,bucket_1,0.0
1,bucket_2,5.0
2,bucket_3,10.0
3,bucket_4,20.0
4,bucket_5,30.0
5,bucket_6,60.0
6,bucket_7,90.0


## OTHERAREA

In [404]:
# HH
df = pd.DataFrame.spatial.from_featureclass('.\Outputs\map\OTHERAREA.gdb\HH_PROJECTIONS_by_OTHERAREA')
jenks_mins(df,'OtherArea')

,jenks,Value
0,bucket_1,0.0
1,bucket_2,850.0
2,bucket_3,2800.0
3,bucket_4,5800.0
4,bucket_5,9900.0
5,bucket_6,14500.0
6,bucket_7,21500.0


In [405]:
# HJI
df = pd.DataFrame.spatial.from_featureclass('.\Outputs\map\OTHERAREA.gdb\HJI_PROJECTIONS_by_OTHERAREA')
jenks_mins(df,'OtherArea')

,jenks,Value
0,bucket_1,1700
1,bucket_2,19000
2,bucket_3,32000
3,bucket_4,44500
4,bucket_5,60500
5,bucket_6,115000
6,bucket_7,165000


In [406]:
# IND
df = pd.DataFrame.spatial.from_featureclass('.\Outputs\map\OTHERAREA.gdb\IND_PROJECTIONS_by_OTHERAREA')
jenks_mins(df, 'OtherArea')

,jenks,Value
0,bucket_1,10
1,bucket_2,1400
2,bucket_3,3200
3,bucket_4,6100
4,bucket_5,10500
5,bucket_6,17000
6,bucket_7,23000


In [407]:
# OTHR
df = pd.DataFrame.spatial.from_featureclass('.\Outputs\map\OTHERAREA.gdb\OTHR_PROJECTIONS_by_OTHERAREA')
jenks_mins(df, 'OtherArea')

,jenks,Value
0,bucket_1,1200
1,bucket_2,8200
2,bucket_3,16500
3,bucket_4,27000
4,bucket_5,42000
5,bucket_6,72000
6,bucket_7,99500


In [408]:
# POP
df = pd.DataFrame.spatial.from_featureclass('.\Outputs\map\OTHERAREA.gdb\POP_PROJECTIONS_by_OTHERAREA')
jenks_mins(df, 'OtherArea')

,jenks,Value
0,bucket_1,0.0
1,bucket_2,2200.0
2,bucket_3,7300.0
3,bucket_4,14500.0
4,bucket_5,25000.0
5,bucket_6,36500.0
6,bucket_7,50000.0


In [409]:
# RTL
df = pd.DataFrame.spatial.from_featureclass('.\Outputs\map\OTHERAREA.gdb\RTL_PROJECTIONS_by_OTHERAREA')
jenks_mins(df, 'OtherArea')

,jenks,Value
0,bucket_1,10
1,bucket_2,1700
2,bucket_3,4200
3,bucket_4,6700
4,bucket_5,10500
5,bucket_6,13500
6,bucket_7,19500


In [410]:
# TPCL
df = pd.DataFrame.spatial.from_featureclass('.\Outputs\map\OTHERAREA.gdb\TPCL_PROJECTIONS_by_OTHERAREA')
jenks_mins(df, 'OtherArea')

,jenks,Value
0,bucket_1,1700
1,bucket_2,11500
2,bucket_3,20000
3,bucket_4,31000
4,bucket_5,45000
5,bucket_6,75500
6,bucket_7,122000


# quantiles

In [411]:
# melted['quantile'] = pd.qcut(melted['Households'].dropna(), duplicates='drop', q=8, labels=['bucket_1', 'bucket_2', 'bucket_3', 'bucket_4', 'bucket_5', 'bucket_6', 'bucket_7', 'bucket_8'])
# melted

In [412]:
# melted.groupby('quantile')[['Households']].min()

# Natural Breaks Method

In [413]:
# data_cols = [c for c in list(df.columns) if c.startswith('N_') |  c.startswith('O_')]
# melted = pd.melt(df, id_vars =['CenterName'], value_vars=data_cols, var_name ='Year', value_name ='Households')
# melted

In [414]:
# breaks = jenkspy.jenks_breaks(melted['Households'], n_classes=8)
# print(breaks)

In [415]:
# melted['cut_jenks'] = pd.cut(melted['Households'],
#                         bins=breaks,
#                         labels=['bucket_1', 'bucket_2', 'bucket_3', 'bucket_4', 'bucket_5', 'bucket_6', 'bucket_7', 'bucket_8'],
#                         include_lowest=True)

In [416]:
# melted.groupby('cut_jenks')[['Households']].min()